# Importing important libraries

In [1]:
import time
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import re

In [108]:
#connect to times of india webpage
sURL="https://timesofindia.indiatimes.com/"
driver=webdriver.Chrome("/Users/bhupendrakumar/Documents/Personal/chromedriver")
driver.get(sURL)

In [3]:
# Navigate to archives page
link=driver.find_element(By.LINK_TEXT,"Archives")
Archives_Address=link.get_attribute('href')
driver.get(Archives_Address)
time.sleep(3)

In [4]:
# Grab data for Feb 2018
links=driver.find_elements(By.LINK_TEXT,"July")

from datetime import date
d=date.today()
ActiveYear=int(d.strftime("%Y"))
YearToBesearched=2020
Link_Index=ActiveYear-YearToBesearched
driver.get(links[Link_Index-1].get_attribute('href'))

In [5]:
#Store links of days
Days_links=[]
for iDay in range(1,32):
    webaddress=driver.find_element(By.LINK_TEXT,str(iDay)).get_attribute('href')
    Days_links.append(webaddress)

In [6]:
#Store news links
News_links=[]
for iDay in Days_links:
    driver.get(iDay)
    time.sleep(2)
    #Set of news links
    News_Addresses=driver.find_elements_by_xpath("//span[@style='font-family:arial ;font-size:12;color: #006699']")
    
    #News links
    Text_Links=News_Addresses[0].text
    texts=Text_Links.split("\n")
    pattern=r"[a-zA-Z]+"
    for sText in texts:    
        if re.search(pattern,sText):
            webaddress=driver.find_element(By.LINK_TEXT,sText).get_attribute('href')
            News_links.append(webaddress)

In [106]:
len(News_links)

7719

## Defining and intializing lists

In [109]:
Headlines_list=[]
Authors_list=[]
Dates_list=[]
News_list=[]
Verticals_list=[]

In [123]:
#Prev i=5870
i=7211
for sNews_Link in News_links[7211:]:
    i+=1
    print(i)
    driver.get(sNews_Link)
    time.sleep(3)
    try:
        # If error class is present and is empty then add the texts of desired classes.
        driver.find_element_by_xpath("//div[@class='error']")
        if driver.find_element_by_xpath("//div[@class='error']").text =="":
            #Extracting Headlines
            try:
                headline=driver.find_element_by_xpath("//h1[@class='_23498']")
                headline=headline.text
            except ElementNotInteractableException:
                headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                headline=headline.text
            except NoSuchElementException:
                try:
                    headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                    headline=headline.text
                except NoSuchElementException:
                    headline="-"
            Headlines_list.append(headline)

            #Extracting Authors and Dates
            try:
                Author_Date=driver.find_element_by_xpath("//div[@class='_3Mkg- byline']").text
                if "|" in Author_Date:
                    Authors_list.append(Author_Date[:Author_Date.index(" | ")])
                else:
                    Authors_list.append("-")

                if "Jul" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Jul"):])

            except ElementNotInteractableException:
                Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Jul")])
                if "Jul" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Jul"):])
            except NoSuchElementException:
                try:
                    Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                    Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                    
                    if "Jul" not in Author_Date:
                        Dates_list.append("-")
                        Authors_list.append("-")
                    else:
                        Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Jul")])
                        Dates_list.append(Author_Date[Author_Date.index("Jul"):])
                except NoSuchElementException:
                    Dates_list.append("-")
                    Authors_list.append("-")
            #Extracting Description
            try:
                News=driver.find_element_by_xpath("//div[@class='ga-headlines']").text
            except ElementNotInteractableException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            except NoSuchElementException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            News_list.append(News)

            #Extracting Verticals
            try:
                Vertical=driver.find_elements_by_xpath("//span[@itemprop='name']")
                if len(Vertical)>=2:
                    Vert=Vertical[1].text
                else:
                    Vert="-"
            except ElementNotInteractableException:
                Vert="-"
            except NoSuchElementException:
                Vert="-"

            Verticals_list.append(Vert)
        else: # If error class exists and text data is peresent then probably news has been removed.
              # Continue from the next page
            continue
    except NoSuchElementException: # If no error class exists, then should have no problem in collecting data
        #Extracting Headlines
            try:
                headline=driver.find_element_by_xpath("//h1[@class='_23498']")
                headline=headline.text
            except ElementNotInteractableException:
                headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                headline=headline.text
            except NoSuchElementException:
                try:
                    headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                    headline=headline.text
                except NoSuchElementException:
                    headline="-"
            Headlines_list.append(headline)

            #Extracting Authors and Dates
            try:
                Author_Date=driver.find_element_by_xpath("//div[@class='_3Mkg- byline']").text
                if "|" in Author_Date:
                    Authors_list.append(Author_Date[:Author_Date.index(" | ")])
                else:
                    Authors_list.append("-")

                if "Jul" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Jul"):])

            except ElementNotInteractableException:
                Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Jul")])
                if "Jul" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Jul"):])
            except NoSuchElementException:
                try:
                    Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                    Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                    
                    if "Jul" not in Author_Date:
                        Authors_list.append("-")
                        Dates_list.append("-")
                    else:
                        Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Jul")])
                        Dates_list.append(Author_Date[Author_Date.index("Jul"):])
                except NoSuchElementException:
                    Dates_list.append("-")
                    Authors_list.append("-")
            #Extracting Description
            try:
                News=driver.find_element_by_xpath("//div[@class='ga-headlines']").text
            except ElementNotInteractableException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            except NoSuchElementException:
                News=driver.find_element_by_xpath("//div[@class='Normal']").text
                News=News.replace("\n","")
            News_list.append(News)

            #Extracting Verticals
            try:
                Vertical=driver.find_elements_by_xpath("//span[@itemprop='name']")
                if len(Vertical)>=2:
                    Vert=Vertical[1].text
                else:
                    Vert="-"
            except ElementNotInteractableException:
                Vert="-"
            except NoSuchElementException:
                Vert="-"

            Verticals_list.append(Vert)

7212
7213
7214
7215
7216
7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
7243
7244
7245
7246
7247
7248
7249
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266
7267
7268
7269
7270
7271
7272
7273
7274
7275
7276
7277
7278
7279
7280
7281
7282
7283
7284
7285
7286
7287
7288
7289
7290
7291
7292
7293
7294
7295
7296
7297
7298
7299
7300
7301
7302
7303
7304
7305
7306
7307
7308
7309
7310
7311
7312
7313
7314
7315
7316
7317
7318
7319
7320
7321
7322
7323
7324
7325
7326
7327
7328
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339
7340
7341
7342
7343
7344
7345
7346
7347
7348
7349
7350
7351
7352
7353
7354
7355
7356
7357
7358
7359
7360
7361
7362
7363
7364
7365
7366
7367
7368
7369
7370
7371
7372
7373
7374
7375
7376
7377
7378
7379
7380
7381
7382
7383
7384
7385
7386
7387
7388
7389
7390
7391
7392
7393
7394
7395
7396
7397
7398
7399
7400
7401
7402
7403
7404
7405
7406
7407
7408
7409
7410
7411


In [126]:
driver.quit()

In [122]:
print(len(News_list))
print(len(Headlines_list))
print(len(Authors_list))
print(len(Dates_list))
print(len(Verticals_list))

1338
1338
1338
1338
1338


## Storing Data in CSV file

In [124]:
#Declaring the dataframe
Scrapped_data=pd.DataFrame()

#Initializing columns in the new data frame
Scrapped_data['Date']=Dates_list
Scrapped_data['Author']=Authors_list
Scrapped_data['Vertical']=Verticals_list
Scrapped_data['Headline']=Headlines_list
Scrapped_data['Description']=News_list

In [125]:
#Storing data in csv file, in m ultiple iterations
#Scrapped_data.to_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/July2020_data5.csv")

In [134]:
# Merge different data files collected during extraction process
Data_1=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/July2020_data.csv")
Data_2=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/July2020_data2.csv")
Data_3=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/July2020_data3.csv")
Data_4=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/July2020_data4.csv")
Data_5=pd.read_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/July2020_data5.csv")

In [135]:
Final_data=pd.concat([Data_1,Data_2,Data_3,Data_4,Data_5],ignore_index=True)

In [136]:
# It has 7712 rows and 6 columns
Final_data.shape

(7712, 6)

In [141]:
Final_data.columns

Index(['Unnamed: 0', 'Date', 'Author', 'Vertical', 'Headline', 'Description'], dtype='object')

In [142]:
# Drop unnamed column
Final_data.drop('Unnamed: 0',inplace=True,axis=1)

In [143]:
# Observing first 20 rows of the collected data
Final_data.head(20)

Date                                 Author  \
0   Jul 1, 2020, 00:00 IST                       TNNVinay Lokesh    
1   Jul 1, 2020, 00:00 IST                TNNJoyeeta Chakravorty    
2   Jul 1, 2020, 00:00 IST        TIMESOFINDIA.COMNupur Amarnath    
3   Jul 1, 2020, 00:00 IST  TNNJoyeeta Chakravorty, Vinay Lokesh    
4   Jul 1, 2020, 00:03 IST              TNNShamayita.Chakraborty    
5   Jul 1, 2020, 00:06 IST              TNNShamayita.Chakraborty    
6   Jul 1, 2020, 00:09 IST              TNNShamayita.Chakraborty    
7   Jul 1, 2020, 00:23 IST                     Sujit kumar bisoyi   
8   Jul 1, 2020, 00:30 IST                                          
9   Jul 1, 2020, 00:32 IST                                   TNN    
10  Jul 1, 2020, 00:40 IST                        M Saleem Pandit   
11  Jul 1, 2020, 00:50 IST                              Updated:    
12  Jul 1, 2020, 00:45 IST                                   TNN    
13                       -                                      -   
14  Jul 1, 2020, 01:00 IST                     TNNNeha Maheshwri    
15                       -                                      -   
16                       -                                      -   
17                       -                                      -   
18  Jul 1, 2020, 01:26 IST                           FLAVIO LOPES   
19  Jul 1, 2020, 12:57 IST              TIMESOFINDIA.COMUpdated:    

                Vertical                                           Headline  \
0          Entertainment  Self-isolation gave Aishani Shetty time to fin...   
1              Lifestyle  Parents worry about digital eye strain among k...   
2              Lifestyle   Rajesh Pratap Singh all set to revamp Satya Paul   
3                     TV  Dubbed content forces Kannada TV actors to tak...   
4          Entertainment  Neeraj Kabi joins hand with Sujoy Prosad in a ...   
5          Entertainment  TikTok was just like any other platform to con...   
6          Entertainment  Upal, Prabuddha, Dipangshu and friends join ha...   
7              CITY NEWS  Odisha: Night curfew relaxed for extra two hou...   
8          Entertainment      Soha Ali Khan makes a 'summertime' resolution   
9              Lifestyle  When you adopt an animal you're actually givin...   
10            INDIA NEWS  Killer of 4-yr boy among two terrorists killed...   
11         Entertainment            Mallika Sherawat is happy 'to be alive'   
12         Entertainment               A special birthday for Allari Naresh   
13             AHMEDABAD                                                  -   
14                    TV  Ashi Singh replaces Avneet Kaur in ‘Aladdin: N...   
15             BANGALORE                                                  -   
16                  PUNE                                                  -   
17  CORONAVIRUS OUTBREAK                                                  -   
18           SPORTS NEWS  We will find best possible options for FC Goa,...   
19         Entertainment  Sonakshi Sinha poses for a cool mirror selfie ...   

                                          Description  
0   Aishani Shetty, who was last seen in Nam Gani ...  
1   The use of technology in our day-to-day life h...  
2   Edward de Bono famously said, “Creativity invo...  
3   It's been a while since many Kannada TV serial...  
4   The powerful actor that he is, Neeraj Kabi has...  
5   A day after 59 Chinese apps, including TikTok,...  
6   This year, the World Music Day celebration was...  
7   BHUBANESWAR: With increasing number of Covid-1...  
8   The Pataudis love their holidays - be it Saif ...  
9   The campaign #AdoptDontShop has been trending ...  
10  SRINAGAR: Two terrorists belonging to the Jamm...  
11  No, nothing has happened to Mallika Sherawat. ...  
12  Allari Naresh’s 38 th birthday on Tuesday was ...  
13  Lockdown 5, better known as Unlock 1, brought ...  
14  It’s barely been a couple of days since the sh...  
15  ‘Effective c

In [145]:
#Store Final data in csv format
Final_data.to_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/July2020_data(TOI).csv")